In [2]:
# import data
import os
import pandas as pd
import re

os.chdir("C:\Users\Moyp\Desktop")

df = pd.read_csv('sample-data.csv')

In [4]:
# quick look at data
df.head()

,id,description
0,1,Active classic boxers - There's a reason why o...
1,2,Active sport boxer briefs - Skinning up Glory ...
2,3,Active sport briefs - These superbreathable no...
3,4,"Alpine guide pants - Skin in, climb ice, switc..."
4,5,"Alpine wind jkt - On high ridges, steep ice an..."


In [31]:
# let's split the name of the item and description into two different columns
test = df['description'][0]

In [25]:
# split by first -
test = test.split(' - ',1)
name = test[0]
name

'Active classic boxers'

In [26]:
test = test[1].split('Details:')
description = test[0]
description

'There\'s a reason why our boxers are a cult favorite - they keep their cool, especially in sticky situations. The quick-drying, lightweight underwear takes up minimal space in a travel pack. An exposed, brushed waistband offers next-to-skin softness, five-panel construction with a traditional boxer back for a classic fit, and a functional fly. Made of 3.7-oz 100% recycled polyester with moisture-wicking performance. Inseam (size M) is 4 1/2". Recyclable through the Common Threads Recycling Program.<br><br><b>'

In [29]:
test = test[1].split('Fabric:')
details = test[0]
fabric = test[1]
print details,fabric

</b><ul> <li>"Silky Capilene 1 fabric is ultralight, breathable and quick-to-dry"</li> <li>"Exposed, brushed elastic waistband for comfort"</li> <li>5-panel construction with traditional boxer back</li> <li>"Inseam (size M) is 4 1/2"""</li></ul><br><br><b>  </b>3.7-oz 100% all-recycled polyester with Gladiodor natural odor control for the garment. Recyclable through the Common Threads Recycling Program<br><br><b>Weight: </b>99 g (3.5 oz)<br><br>Made in Mexico.


In [30]:
def split_description(string):
    # name
    string_split = string.split(' - ',1)
    name = string_split[0]
    
    # description
    string_split = string_split[1].split('Details:')
    description = string_split[0]
    
    # details and fabric
    string_split = string_split[1].split('Fabric:')
    details = string_split[0]
    fabric = string_split[1]
    
    return(name, description, details, fabric)

In [33]:
test1 = split_description(test)

In [105]:
#remove special characters and extra white spaces 
test2 = [re.sub('</?\w*>|[\W]|[\d]',' ',test) for test in test1]
test2 = [' '.join(test.split()) for test in test2]
test2

['Active classic boxers',
 'There s a reason why our boxers are a cult favorite they keep their cool especially in sticky situations The quick drying lightweight underwear takes up minimal space in a travel pack An exposed brushed waistband offers next to skin softness five panel construction with a traditional boxer back for a classic fit and a functional fly Made of oz recycled polyester with moisture wicking performance Inseam size M is Recyclable through the Common Threads Recycling Program',
 'Silky Capilene fabric is ultralight breathable and quick to dry Exposed brushed elastic waistband for comfort panel construction with traditional boxer back Inseam size M is',
 'oz all recycled polyester with Gladiodor natural odor control for the garment Recyclable through the Common Threads Recycling Program Weight g oz Made in Mexico']